In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [7]:
requested_page = requests.get('https://www.yelp.com/biz/aya-homes-san-francisco')
soup = BeautifulSoup(requested_page.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [8]:
reviews

['My husband and I worked with Shawn Ajdari on an extensive renovation: converting the existing attic into a living space (including structural reinforcement); adding a new stairway, dormers, and windows; drywall installation; installing lights, outlets, and ceiling fans; roof replacement; finish carpentry; and wood flooring. Shawn and his team were responsive, worked with our timelines (including splitting the renovation into 2 phases), and performed high quality and timely work. Shawn worked with our architect and structural engineer to ensure plans were complete and all inspections were passed. I would absolutely work with Shawn again and have already recommended him to friends.',
 'Shawn and his team stripped down our kitchen and pantry to replace all the cabinetry and counters, replaced vanities in three bathrooms and handled flood remediation which happened while they were working on our kitchen.As with most remodels, you open a wall and new problems arise. Add in the flood damag

In [10]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [11]:
df

,review
0,My husband and I worked with Shawn Ajdari on a...
1,Shawn and his team stripped down our kitchen a...
2,I would not recommend a contractor who gives l...
3,HiI tried to help by understanding your reques...
4,Shawn recently completed a leaking roof deck a...
5,"AYA Homes did a pretty extensive remodel, addi..."
6,Patrick thank you for your great reviews. We s...
7,"No one likes to be around when a ""once in a 10..."
8,AYA is owned and operated by Shawn who has bee...
9,"Shawn and the team showed up on time, setup a ..."


In [16]:
def find_sentiment(review):
  sentiment_mapping = {
    1: "Very Negative",
    2: "Negative",
    3: "Neutral",
    4: "Positive",
    5: "Very Positive"
    }
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  sentiment_label_index = int(torch.argmax(result.logits)) + 1
  return sentiment_mapping[sentiment_label_index]

In [17]:
df['sentiment'] = df['review'].apply(lambda x: find_sentiment(x[:512]))

In [18]:
df

,review,sentiment
0,My husband and I worked with Shawn Ajdari on a...,Very Positive
1,Shawn and his team stripped down our kitchen a...,Very Positive
2,I would not recommend a contractor who gives l...,Very Negative
3,HiI tried to help by understanding your reques...,Neutral
4,Shawn recently completed a leaking roof deck a...,Positive
5,"AYA Homes did a pretty extensive remodel, addi...",Very Positive
6,Patrick thank you for your great reviews. We s...,Very Positive
7,"No one likes to be around when a ""once in a 10...",Very Negative
8,AYA is owned and operated by Shawn who has bee...,Very Positive
9,"Shawn and the team showed up on time, setup a ...",Very Positive
